In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

In [2]:
ws = Workspace.from_config()
def_blob_store = Datastore(ws, "workspaceblobstore")

In [3]:
source_directory = 'script'

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpucluster"

try:
    aml_compute = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS1_V2',
                                                           max_nodes=5)
    aml_compute = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [5]:
blob_input_data = DataReference(
    datastore=def_blob_store,
    data_reference_name="raw_input_data",
    path_on_datastore="raw_dataset")

# Building Pipeline Handled Data Step

In [6]:
handled_data = PipelineData("handled_data",datastore=def_blob_store)

In [7]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow','pandas','scikit-learn'])

In [27]:
run_config.save(path='./temp.json')

In [28]:
HandledInputStep = PythonScriptStep(
    script_name="input_handle.py", 
    arguments=["--raw_input_data", blob_input_data, "--handled_data", handled_data],
    inputs=[blob_input_data],
    outputs=[handled_data],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

# Building Pipeline Logistic Train Step

In [29]:
logistic_trained_models = PipelineData("logistic_trained_models",datastore=def_blob_store)

In [30]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow','pandas','scikit-learn'])

In [31]:
LogisticTrainStep = PythonScriptStep(
    script_name="logistic_train.py", 
    arguments=["--handled_data", handled_data, "--logistic_trained_models", logistic_trained_models],
    inputs=[handled_data],
    outputs=[logistic_trained_models],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

# Building Pipeline Neural Network Train Step

In [32]:
nn_trained_models = PipelineData("nn_trained_models",datastore=def_blob_store)

In [33]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow','pandas','scikit-learn'])

In [34]:
NNTrainStep = PythonScriptStep(
    script_name="nn_train.py", 
    arguments=["--handled_data", handled_data, "--nn_trained_models", nn_trained_models],
    inputs=[handled_data],
    outputs=[nn_trained_models],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

# Building Pipeline Select Model Step

In [35]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow','pandas','scikit-learn'])

In [36]:
SelectModelStep = PythonScriptStep(
    script_name="select_model.py", 
    arguments=["--handled_data", handled_data, "--nn_trained_models", nn_trained_models, "--logistic_trained_models", logistic_trained_models],
    inputs=[handled_data,nn_trained_models,logistic_trained_models],
    compute_target=aml_compute, 
    source_directory=source_directory,
    runconfig=run_config
)

In [37]:
pipeline1 = Pipeline(workspace=ws, steps=[HandledInputStep,LogisticTrainStep,NNTrainStep,SelectModelStep])

In [38]:
pipeline_run1 = Experiment(ws, 'Train_Multi_Model').submit(pipeline1)
RunDetails(pipeline_run1).show()

ClientRequestError: Error occurred in request., RetryError: HTTPSConnectionPool(host='southeastasia.aether.ms', port=443): Max retries exceeded with url: /api/v1.0/subscriptions/0b079c47-7fbc-419d-951e-5935de7a5f08/resourceGroups/nghiadev/providers/Microsoft.MachineLearningServices/workspaces/machine-learning-workspace-nghia/DataTypes (Caused by ResponseError('too many 500 error responses',))

In [ ]:
pipeline_run1.wait_for_completion(show_output=True)
cpu_cluster.delete()

In [39]:
published_pipeline1 = pipeline1.publish(name="Train_Choose_Pipeline", description="Vu Trung Nghia", continue_on_step_failure=True)

ClientRequestError: Error occurred in request., RetryError: HTTPSConnectionPool(host='southeastasia.aether.ms', port=443): Max retries exceeded with url: /api/v1.0/subscriptions/0b079c47-7fbc-419d-951e-5935de7a5f08/resourceGroups/nghiadev/providers/Microsoft.MachineLearningServices/workspaces/machine-learning-workspace-nghia/DataTypes (Caused by ResponseError('too many 500 error responses',))